In [2]:
import pandas as pd

data_path = './data/car-sim1/'
img_path = './data/car-sim1/IMG/'
df = pd.read_csv(data_path+'driving_log.csv', header=None)

df.head()

,0,1,2,3,4,5,6
0,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000006
1,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000012
2,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000006
3,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000004
4,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000010


In [3]:
import csv
import cv2
import numpy as np

lines = []
with open(data_path+'driving_log.csv') as csvFile:
    reader = csv.reader(csvFile)
    #next(reader, None)
    for line in reader:
        lines.append(line)
        
def process_image(img): 
    # preprocess input
    return img


In [3]:
images = []
measurements = []
for line in lines:
    '''
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = img_path + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    '''
    
    steering_center = float(line[3])

    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # read in images from center, left and right cameras
    filename = line[0].split('/')[-1]
    img_center = process_image(cv2.imread(img_path + filename))
    filename = line[1].split('/')[-1]
    img_left = process_image(cv2.imread(img_path + filename))
    filename = line[2].split('/')[-1]
    img_right = process_image(cv2.imread(img_path + filename))

    # add images and angles to data set
    images.extend([img_center, img_left, img_right])
    measurements.extend([steering_center, steering_left, steering_right])
    
X_train = np.array(images)
Y_train = np.array(measurements)

print('X_train:', X_train.shape)
print('Y_train:', Y_train.shape)

X_train: (6861, 160, 320, 3)
Y_train: (6861,)


In [44]:
images_center = []
images_left = []
images_right = []
measurements_center = []
measurements_left = []
measurements_right = []
for line in lines:

    steering_center = float(line[3])

    # create adjusted steering measurements for the side camera images
    correction = 0.25 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # read in images from center, left and right cameras
    filename = line[0].split('/')[-1]
    img_center = process_image(cv2.imread(img_path + filename))
    
    filename = line[1].split('/')[-1]
    img_left = process_image(cv2.imread(img_path + filename))
    
    filename = line[2].split('/')[-1]
    img_right = process_image(cv2.imread(img_path + filename))
    
    # add images and angles to data set
    images_center.append(img_center)
    images_left.append(img_left)
    images_right.append(img_right)
    
    measurements_center.append(steering_center)
    measurements_left.append(steering_left)
    measurements_right.append(steering_right)
    
# convert to array
images_center = np.array(images_center)
images_left = np.array(images_left)
images_right = np.array(images_right)

measurements_center = np.array(measurements_center)
measurements_left = np.array(measurements_left)
measurements_right = np.array(measurements_right)
    
print('images:', images_center.shape)
print('measurements:', measurements_center.shape)

images: (2287, 160, 320, 3)
measurements: (2287,)


## Split data and augment

In [45]:
# split 70/15/15
length = len(measurements_center)
train_size = int(length * 0.7)

indices = np.random.permutation(length)
train_idx, test_idx = indices[:train_size], indices[train_size:]

val_size = int(len(test_idx) * 0.5)
val_idx, test_idx = test_idx[:val_size], test_idx[val_size:]

print('train_idx:', len(train_idx))
print('val_idx:', len(val_idx))
print('test_idx:', len(test_idx))

images_train = np.concatenate((
    images_center[train_idx],
    images_left[train_idx],
    images_right[train_idx],
    images_left[val_idx],
    images_right[val_idx],
    images_left[test_idx], 
    images_right[test_idx]
))
measurements_train = np.concatenate((
    measurements_center[train_idx], 
    measurements_left[train_idx], 
    measurements_right[train_idx], 
    measurements_left[val_idx], 
    measurements_right[val_idx], 
    measurements_left[test_idx], 
    measurements_right[test_idx]
))

images_val, measurements_val = images_center[val_idx], measurements_center[val_idx]
images_test, measurements_test = images_center[test_idx], measurements_center[test_idx]

# augmented train set
augmented_images = []
augmented_measurements = []
for image, measurement in zip(images_train, measurements_train):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
images_train = np.array(augmented_images)
measurements_train = np.array(augmented_measurements)
    
    
print('images_train:', images_train.shape)
print('measurements_train:', measurements_train.shape)
print('images_val:', images_val.shape)
print('measurements_val:', measurements_val.shape)
print('images_test:', images_test.shape)
print('measurements_test:', measurements_test.shape)

train_idx: 1600
val_idx: 343
test_idx: 344
images_train: (12348, 160, 320, 3)
measurements_train: (12348,)
images_val: (343, 160, 320, 3)
measurements_val: (343,)
images_test: (344, 160, 320, 3)
measurements_test: (344,)


In [102]:
X_train = images_train
Y_train = measurements_train

print('X_train:', X_train.shape)
print('Y_train:', Y_train.shape)

X_train: (9600, 160, 320, 3)
Y_train: (9600,)


## First Model

In [33]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.optimizers import Adam

model = Sequential()
#model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(lr=0.0001))
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, epochs=2)

model.save('model.h5')

Train on 10972 samples, validate on 2744 samples
Epoch 1/2
10972/10972 [==============================] - 7s 677us/step - loss: 0.0980 - val_loss: 0.1220
Epoch 2/2
10972/10972 [==============================] - 7s 623us/step - loss: 0.0578 - val_loss: 0.1509


## Second Model

In [49]:
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Flatten, Dense, Lambda, Cropping2D

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5))
model.add(Conv2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(lr=0.001))
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, epochs=3)

model.save('model.h5')

Train on 10977 samples, validate on 2745 samples
Epoch 1/3
10977/10977 [==============================] - 10s 889us/step - loss: 0.5664 - val_loss: 0.0745
Epoch 2/3
10977/10977 [==============================] - 9s 796us/step - loss: 0.0437 - val_loss: 0.0645
Epoch 3/3
10977/10977 [==============================] - 9s 803us/step - loss: 0.0374 - val_loss: 0.0639


## Third Model

In [9]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers.pooling import GlobalMaxPooling2D
from keras.models import Model
from tqdm import tqdm

modelTransfer = VGG16(weights='imagenet', include_top=False, pooling='max')

#x = modelTransfer.output
#x = GlobalMaxPooling2D()(x)
#net = Model(inputs=modelTransfer.input, outputs=x)
modelTransfer.summary()

def get_bottleneck_feature(img):
    # convert 3D tensor to 4D tensor with shape (1, ?, ?, 3) and return 4D tensor
    x = np.expand_dims(img, axis=0)
    
    # preprocess input
    x = preprocess_input(x)
    
    return modelTransfer.predict(x)

def get_bottleneck_features(images):
    bottleneck_features = []

    for img in tqdm(images):
        bottleneck_features.append(get_bottleneck_feature(img))
        #try:
        #    bottleneck_features.append(bottleneck_feature(img))
        #except OSError:
        #    None 
            
    return np.vstack(bottleneck_features)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [10]:
import pickle

X_train = get_bottleneck_features(images_train)
Y_train = measurements_train

X_val = get_bottleneck_features(images_val)
Y_val = measurements_val

X_test = get_bottleneck_features(images_test)
Y_test = measurements_test

bottleneck_feature = {
    'X_train': X_train,
    'Y_train': Y_train,
    'X_val': X_val,
    'Y_val': Y_val,
    'X_test': X_test,
    'Y_test': Y_test
}

with open('models/bottleneck_feature.pickle', 'wb') as f:
    pickle.dump(bottleneck_feature, f, pickle.HIGHEST_PROTOCOL)
    
print('X_train:', X_train.shape)
print('X_val:', X_val.shape)
print('X_test:', X_test.shape)

100%|██████████| 344/344 [00:02<00:00, 154.35it/s]

X_train: (9600, 512)
X_val: (343, 512)
X_test: (344, 512)


In [11]:
import pickle

with open('models/bottleneck_feature.pickle', 'rb') as f:
    bottleneck_feature = pickle.load(f)
    
X_train = bottleneck_feature['X_train']
Y_train = bottleneck_feature['Y_train']

X_val = bottleneck_feature['X_val']
Y_val = bottleneck_feature['Y_val']

X_test = bottleneck_feature['X_test']
Y_test = bottleneck_feature['Y_test']

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

(9600, 512)
(9600,)
(343, 512)
(343,)
(344, 512)
(344,)


In [12]:
from keras.layers import Dense, Flatten, BatchNormalization
from keras.models import Sequential
from keras.regularizers import l1, l2

model = Sequential()
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.02), input_shape=X_train.shape[1:]))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.02)))
model.add(Dense(1))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 197,377
Trainable params: 197,377
Non-trainable params: 0
_________________________________________________________________


In [13]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint 
checkpointer = ModelCheckpoint(filepath='models/weights.best.hdf5', 
                               verbose=1, save_best_only=True)

model.compile(loss='mse', optimizer=Adam(lr=0.0001))
model.fit(X_train, Y_train, 
          validation_data=(X_val, Y_val),
          shuffle=True, epochs=100, callbacks=[checkpointer], verbose=1)

model.save('models/model.h5')

Train on 9600 samples, validate on 343 samples
Epoch 1/100
9600/9600 [==============================] - 1s 140us/step - loss: 68.0117 - val_loss: 30.8114

Epoch 00001: val_loss improved from inf to 30.81140, saving model to models/weights.best.hdf5
Epoch 2/100
9600/9600 [==============================] - 1s 76us/step - loss: 23.0870 - val_loss: 20.5189

Epoch 00002: val_loss improved from 30.81140 to 20.51890, saving model to models/weights.best.hdf5
Epoch 3/100
9600/9600 [==============================] - 1s 75us/step - loss: 17.9834 - val_loss: 18.1462

Epoch 00003: val_loss improved from 20.51890 to 18.14622, saving model to models/weights.best.hdf5
Epoch 4/100
9600/9600 [==============================] - 1s 72us/step - loss: 15.2971 - val_loss: 15.3619

Epoch 00004: val_loss improved from 18.14622 to 15.36188, saving model to models/weights.best.hdf5
Epoch 5/100
9600/9600 [==============================] - 1s 77us/step - loss: 13.8874 - val_loss: 14.3798

Epoch 00005: val_loss impr

9600/9600 [==============================] - 1s 73us/step - loss: 2.2174 - val_loss: 2.1081

Epoch 00041: val_loss improved from 2.31903 to 2.10811, saving model to models/weights.best.hdf5
Epoch 42/100
9600/9600 [==============================] - 1s 74us/step - loss: 2.0163 - val_loss: 1.9206

Epoch 00042: val_loss improved from 2.10811 to 1.92058, saving model to models/weights.best.hdf5
Epoch 43/100
9600/9600 [==============================] - 1s 73us/step - loss: 1.8370 - val_loss: 1.7780

Epoch 00043: val_loss improved from 1.92058 to 1.77797, saving model to models/weights.best.hdf5
Epoch 44/100
9600/9600 [==============================] - 1s 76us/step - loss: 1.6680 - val_loss: 1.6094

Epoch 00044: val_loss improved from 1.77797 to 1.60936, saving model to models/weights.best.hdf5
Epoch 45/100
9600/9600 [==============================] - 1s 73us/step - loss: 1.5159 - val_loss: 1.4516

Epoch 00045: val_loss improved from 1.60936 to 1.45159, saving model to models/weights.best.hdf

Epoch 82/100
9600/9600 [==============================] - 1s 71us/step - loss: 0.1215 - val_loss: 0.1267

Epoch 00082: val_loss improved from 0.14077 to 0.12665, saving model to models/weights.best.hdf5
Epoch 83/100
9600/9600 [==============================] - 1s 74us/step - loss: 0.1141 - val_loss: 0.1167

Epoch 00083: val_loss improved from 0.12665 to 0.11665, saving model to models/weights.best.hdf5
Epoch 84/100
9600/9600 [==============================] - 1s 76us/step - loss: 0.1062 - val_loss: 0.1189

Epoch 00084: val_loss did not improve from 0.11665
Epoch 85/100
9600/9600 [==============================] - 1s 74us/step - loss: 0.0979 - val_loss: 0.1072

Epoch 00085: val_loss improved from 0.11665 to 0.10721, saving model to models/weights.best.hdf5
Epoch 86/100
9600/9600 [==============================] - 1s 75us/step - loss: 0.0933 - val_loss: 0.0981

Epoch 00086: val_loss improved from 0.10721 to 0.09807, saving model to models/weights.best.hdf5
Epoch 87/100
9600/9600 [=======

In [14]:
from keras.models import load_model

model = load_model('models/model.h5')
model.load_weights('models/weights.best.hdf5')
model.save('models/model.best.h5')

#model.evaluate(X_test, Y_test)

## Fourth model

In [40]:
X_train = images_train
Y_train = measurements_train

X_val = images_val
Y_val = measurements_val

X_test = images_test
Y_test = measurements_test

In [49]:
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPooling2D
from keras.layers import Flatten, Dense, Lambda, Cropping2D, BatchNormalization, Dropout, AlphaDropout
from keras.models import Sequential

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(BatchNormalization())

model.add(Conv2D(10, (5,5), activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(20, (5,5), activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(30, (5,5), activation='relu'))
#model.add(Dropout(0.5))

model.add(GlobalMaxPooling2D())
model.add(Dense(512))
model.add(Dense(256))
model.add(Dense(1))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_9 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 90, 320, 3)        12        
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 86, 316, 10)       760       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 43, 158, 10)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 39, 154, 20)       5020      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 19, 77, 20)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 15, 73, 30)        15030     
__________

In [50]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint 
checkpointer = ModelCheckpoint(filepath='models/weights.best.hdf5', 
                               verbose=1, save_best_only=True)

model.compile(loss='mse', optimizer=Adam(lr=0.001))
model.fit(X_train, Y_train, 
          validation_data=(X_val, Y_val),
          shuffle=True, epochs=20, callbacks=[checkpointer], verbose=1)

model.save('models/model.h5')

Train on 12348 samples, validate on 343 samples
Epoch 1/20
12348/12348 [==============================] - 12s 932us/step - loss: 0.1417 - val_loss: 0.0277

Epoch 00001: val_loss improved from inf to 0.02772, saving model to models/weights.best.hdf5
Epoch 2/20
12348/12348 [==============================] - 10s 845us/step - loss: 0.0915 - val_loss: 0.0364

Epoch 00002: val_loss did not improve from 0.02772
Epoch 3/20
12348/12348 [==============================] - 10s 844us/step - loss: 0.0644 - val_loss: 0.0367

Epoch 00003: val_loss did not improve from 0.02772
Epoch 4/20
12348/12348 [==============================] - 10s 848us/step - loss: 0.0582 - val_loss: 0.0365

Epoch 00004: val_loss did not improve from 0.02772
Epoch 5/20
12348/12348 [==============================] - 11s 851us/step - loss: 0.0534 - val_loss: 0.0296

Epoch 00005: val_loss did not improve from 0.02772
Epoch 6/20
12348/12348 [==============================] - 10s 847us/step - loss: 0.0508 - val_loss: 0.0356

Epoch 0

In [51]:
from keras.models import load_model

model = load_model('models/model.h5')
#model.load_weights('models/weights.best.hdf5')
model.save('models/model.best.h5')

model.evaluate(X_test, Y_test)

344/344 [==============================] - 0s 1ms/step


0.04487496146629023